Unravelling with the V-model with just one bath

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from _utils import *
from FCS import *

In [5]:
# Define system parameters
N = 3 # Number of levels
nu = 1 # Frequency of the system
a = 0.01 
delta = 0.01
# alpha = 0.5 # No right bath for now
betaL = 1/4
# betaR = 1/3.99 # No right bath for now

# Define the parameters for the V-model
params = {'nu': nu, 'a': a, 'delta': delta, 'beta': betaL}

# Set up the open system problem with one bath at temperature 1/betaL
g_ops, H_uni, c_ops_uni = V_model_1bath(**params, kind='unified')
g_ops, H_sec, c_ops_sec = V_model_1bath(**params, kind='secular')

# Define the steady state density operators
rho_ss_uni = steadystate(H_uni, c_ops_uni)
rho_ss_sec = steadystate(H_sec, c_ops_sec)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[3.90991315e-01+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j]
 [0.00000000e+00+0.00000000e+00j 3.04504342e-01+0.00000000e+00j
  1.08412737e-16+2.48264819e-17j]
 [0.00000000e+00+0.00000000e+00j 1.08412737e-16-2.48264819e-17j
  3.04504342e-01+0.00000000e+00j]]